In [12]:
import numpy as np
import pandas as pd 
df= pd.read_csv("diabetes.csv")
#extract x i.e. all rows, all columns except 'Outcome'
x= df.iloc[:,:-1].values

#extract y i.e. all rows, only last column
y= df.iloc[:,-1].values



In [1]:
import numpy as np
import pandas as pd 
df= pd.read_csv("diabetes.csv")

In [2]:
#extract x i.e. all rows, all columns except 'Outcome'
x= df.iloc[:,:-1].values

#extract y i.e. all rows, only last column
y= df.iloc[:,-1].values

In [3]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
x= scaler.fit_transform(x)

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [5]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

2024-07-26 23:18:42.579235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import keras_tuner as kt

In [13]:
def build_model(hp):
    model=Sequential()
    counter=0

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0 :
            #FIRST LAYER
            model.add(Dense(hp.Int('units'  + str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu','tanh']),
                      input_dim=8))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            model.add(Dense(hp.Int('units'  + str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation' +str(i), values=['relu','tanh'])))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        counter+=1

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['adam','rmsprop','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [14]:
Tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='Tuning2')

In [15]:
Tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 10s


In [16]:
Tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 72,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'relu',
 'dropout1': 0.2,
 'units2': 128,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 128,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 96,
 'activation4': 'tanh',
 'dropout4': 0.4,
 'units5': 32,
 'activation5': 'tanh',
 'dropout5': 0.2,
 'units6': 88,
 'activation6': 'relu',
 'dropout6': 0.2}

In [17]:
model=Tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4849 - accuracy: 0.7492 - val_loss: 0.4693 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4932 - accuracy: 0.7590 - val_loss: 0.4712 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4781 - accuracy: 0.7671 - val_loss: 0.4676 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4882 - accuracy: 0.7736 - val_loss: 0.4651 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4699 - accuracy: 0.7671 - val_loss: 0.4608 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4694 - accuracy: 0.7720 - val_loss: 0.4579 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4775 - accuracy: 0.7590 - val_loss: 0.4569 - val_accuracy: 0.8052
Epoch 14